In [1]:
import pandas as pd
import numpy as np

# Gerar dataset para cada estado

In [2]:
datasetTotal = pd.read_csv('datasetTotal.csv')

In [3]:
listaUF = datasetTotal['UF'].unique()

In [4]:
datasetTotal.head(10)

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
0,0,Alta Floresta D'Oeste (RO),RO,Total,6880,3520,3360,4011,1951,2060,2869,1569,1300,altaflorestadoeste,1,Alta Floresta dOeste
1,1,NaN,RO,15 a 17 anos,1577,830,747,878,438,440,699,392,307,altaflorestadoeste,1,Alta Floresta dOeste
2,2,NaN,RO,18 ou 19 anos,1011,532,479,604,301,303,407,231,176,altaflorestadoeste,1,Alta Floresta dOeste
3,3,NaN,RO,20 a 24 anos,2200,1105,1095,1303,632,671,897,473,424,altaflorestadoeste,1,Alta Floresta dOeste
4,4,NaN,RO,25 a 29 anos,2092,1053,1039,1226,580,646,866,473,393,altaflorestadoeste,1,Alta Floresta dOeste
5,5,Ariquemes (RO),RO,Total,26480,13096,13384,23011,11284,11727,3469,1812,1657,ariquemes,2,Ariquemes
6,6,NaN,RO,15 a 17 anos,5497,2758,2739,4641,2310,2331,856,448,408,ariquemes,2,Ariquemes
7,7,NaN,RO,18 ou 19 anos,3544,1721,1823,3096,1477,1619,448,244,204,ariquemes,2,Ariquemes
8,8,NaN,RO,20 a 24 anos,8981,4456,4525,7920,3908,4012,1061,548,513,ariquemes,2,Ariquemes
9,9,NaN,RO,25 a 29 anos,8458,4161,4297,7354,3589,3765,1104,572,532,ariquemes,2,Ariquemes


In [5]:
datasetDict = {}
for i in listaUF:
  datasetDict[i] = datasetTotal.loc[(datasetTotal['UF']==i) & (datasetTotal['Idade']=='Total')]

Alguns municípios não possuem todos os dados

In [6]:
datasetTotal.loc[(datasetTotal['UF']=='CE') & (datasetTotal['ID']==947)]

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
4715,4715,Eusébio (CE),CE,Total,13839,6814,7025,13839,6814,7025,0,0,0,eusebio,947,Eusébio
4716,4716,NaN,CE,15 a 17 anos,2871,1423,1448,2871,1423,1448,0,0,0,eusebio,947,Eusébio
4717,4717,NaN,CE,18 ou 19 anos,1750,875,875,1750,875,875,0,0,0,eusebio,947,Eusébio
4718,4718,NaN,CE,20 a 24 anos,4812,2367,2445,4812,2367,2445,0,0,0,eusebio,947,Eusébio
4719,4719,NaN,CE,25 a 29 anos,4406,2149,2257,4406,2149,2257,0,0,0,eusebio,947,Eusébio


In [8]:
nova_coluna = []
name_id = "43"     #População Jovem Urbana: Total
year_id = "9"      #Ano: 2010
for index, row in datasetDict['CE'].iterrows():
  region_id = datasetDict['CE'].loc[index,'ID']
  jhi_value = datasetDict['CE'].loc[index,'Urbana_Total']
  if jhi_value == 0:
    print('nulo')
  else:
    sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES ('NULL','" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "');"
    nova_coluna.append(sql)

Antiga função para gerar nova coluna com sqls, tratando os nulos

In [9]:
#def gerar_SQL(name, name_id, year_id, datasetUF):
#  #name = 'Rural_Total'
#  #name_id = "40"     #População Jovem Rural: Total
#  #year_id = "9"      #Ano: 2010
#  nova_coluna = []
#  string = ""
#  count = 0
#  for index, row in datasetUF.iterrows():
#    region_id = datasetUF.loc[index,'ID']
#    jhi_value = datasetUF.loc[index,name]
#    if jhi_value == "-":
#      count += 1
#    else:
#      sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES (NULL,'" + str(region_id) + "','" + name_id + #"','" + year_id + "','" + str(jhi_value) + "');"
#      nova_coluna.append(sql)
#
#  print("O estado " + datasetUF.loc[index,'UF'] + " tem " + str(count) + " valores nulos para o indicador " + name)
#  return nova_coluna

Nova função para gerar lista de SQLs, tratando os nulos. Gera apenas o que vem depois do VALUE

In [9]:
def gerar_values(name, name_id, year_id, datasetUF):
    #name = 'Rural_Total'
    #name_id = "40"     #População Jovem Rural: Total
    #year_id = "9"      #Ano: 2010
    nova_coluna = []
    count = 0
    
    for index, row in datasetUF.iterrows():
        region_id = datasetUF.loc[index,'ID']
        jhi_value = datasetUF.loc[index,name]
        if jhi_value == 0:
            count += 1
        else:
            #sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES (NULL,'" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "');"
            sql = "(NULL,'" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "')"
            nova_coluna.append(sql)        

    print("O estado " + datasetUF.loc[index,'UF'] + " tem " + str(count) + " valores nulos para o indicador " + name)
    return nova_coluna

Nova função para gerar lista de SQL completo, aqui os nulos já estão tratados. Gera um insert into para cada 10 values

In [11]:
def gerar_SQL(lista):
    nova_coluna = []
    auxStr = ""
    string = "INSERT INTO `MuniEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES "
    count = 1 #contador para cada 10
    
    #nova_coluna.append(string)
    auxStr += string
    
    for item in lista:
        if (count%10 == 0):
            #sql = item + "; \n"
            #nova_coluna.append(sql)
            #nova_coluna.append(string) 
            
            auxStr += item + "; "
            nova_coluna.append(auxStr) 
            auxStr = string
            
        else:
            auxStr += item + ", "
            #nova_coluna.append(sql)
        count +=1

    if auxStr.endswith(', '):
        auxStr = auxStr[:-2]
        auxStr += "; "
        nova_coluna.append(auxStr) 
    
    return nova_coluna

In [12]:
name = 'Urbana_Total'
name_id = "43"     #População Jovem Rural: Total
year_id = "9"      #Ano: 2010

In [13]:
teste = gerar_values(name, name_id, year_id, datasetDict['CE'])

O estado CE tem 0 valores nulos para o indicador Urbana_Total


In [14]:
teste

["(NULL,'892','43','9','1207')",
 "(NULL,'893','43','9','2292')",
 "(NULL,'894','43','9','8507')",
 "(NULL,'895','43','9','6467')",
 "(NULL,'896','43','9','1025')",
 "(NULL,'897','43','9','959')",
 "(NULL,'898','43','9','1221')",
 "(NULL,'899','43','9','2256')",
 "(NULL,'900','43','9','4788')",
 "(NULL,'901','43','9','1211')",
 "(NULL,'902','43','9','1624')",
 "(NULL,'903','43','9','19710')",
 "(NULL,'904','43','9','12463')",
 "(NULL,'905','43','9','3802')",
 "(NULL,'906','43','9','1186')",
 "(NULL,'907','43','9','3413')",
 "(NULL,'908','43','9','1103')",
 "(NULL,'909','43','9','1047')",
 "(NULL,'910','43','9','3105')",
 "(NULL,'911','43','9','3239')",
 "(NULL,'912','43','9','907')",
 "(NULL,'913','43','9','2558')",
 "(NULL,'914','43','9','11044')",
 "(NULL,'915','43','9','2308')",
 "(NULL,'916','43','9','3536')",
 "(NULL,'917','43','9','2662')",
 "(NULL,'918','43','9','6674')",
 "(NULL,'919','43','9','5868')",
 "(NULL,'920','43','9','3692')",
 "(NULL,'921','43','9','6940')",
 "(NULL,'

In [18]:
a = gerar_SQL(teste)
for i in a:
    print(i)

INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'892','43','9','4552'), (NULL,'893','43','9','7982'), (NULL,'894','43','9','28242'), (NULL,'895','43','9','25228'), (NULL,'896','43','9','3951'), (NULL,'897','43','9','3448'), (NULL,'898','43','9','4957'), (NULL,'899','43','9','8041'), (NULL,'900','43','9','15947'), (NULL,'901','43','9','4999'); 
INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'902','43','9','5772'), (NULL,'903','43','9','67083'), (NULL,'904','43','9','44035'), (NULL,'905','43','9','13737'), (NULL,'906','43','9','4906'), (NULL,'907','43','9','12733'), (NULL,'908','43','9','3769'), (NULL,'909','43','9','3879'), (NULL,'910','43','9','11952'), (NULL,'911','43','9','11825'); 
INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'912','43','9','3304'), (NULL,'913','43','9','8753'), (NULL,'914','43','9','38022'), (NULL,'915

In [14]:
listaUF

array(['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN',
       'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC',
       'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

In [17]:
name = 'Urbana_Total'
name_id = "43"     #População Jovem Urbana: Total
year_id = "9"      #Ano: 2010

In [15]:
file_name = "Novas/Urbana_Total.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Urbana_Total
O estado AC tem 0 valores nulos para o indicador Urbana_Total
O estado AM tem 0 valores nulos para o indicador Urbana_Total
O estado RR tem 0 valores nulos para o indicador Urbana_Total
O estado PA tem 0 valores nulos para o indicador Urbana_Total
O estado AP tem 0 valores nulos para o indicador Urbana_Total
O estado TO tem 0 valores nulos para o indicador Urbana_Total
O estado MA tem 0 valores nulos para o indicador Urbana_Total
O estado PI tem 0 valores nulos para o indicador Urbana_Total
O estado CE tem 0 valores nulos para o indicador Urbana_Total
O estado RN tem 0 valores nulos para o indicador Urbana_Total
O estado PB tem 0 valores nulos para o indicador Urbana_Total
O estado PE tem 0 valores nulos para o indicador Urbana_Total
O estado AL tem 0 valores nulos para o indicador Urbana_Total
O estado SE tem 0 valores nulos para o indicador Urbana_Total
O estado BA tem 0 valores nulos para o indicador Urbana_Total
O estado

In [16]:
for item in listaUF:
    file_name = "Urbana_Total/" + item +  "_Urbana_Total.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Urbana_Total
O estado AC tem 0 valores nulos para o indicador Urbana_Total
O estado AM tem 0 valores nulos para o indicador Urbana_Total
O estado RR tem 0 valores nulos para o indicador Urbana_Total
O estado PA tem 0 valores nulos para o indicador Urbana_Total
O estado AP tem 0 valores nulos para o indicador Urbana_Total
O estado TO tem 0 valores nulos para o indicador Urbana_Total
O estado MA tem 0 valores nulos para o indicador Urbana_Total
O estado PI tem 0 valores nulos para o indicador Urbana_Total
O estado CE tem 0 valores nulos para o indicador Urbana_Total
O estado RN tem 0 valores nulos para o indicador Urbana_Total
O estado PB tem 0 valores nulos para o indicador Urbana_Total
O estado PE tem 0 valores nulos para o indicador Urbana_Total
O estado AL tem 0 valores nulos para o indicador Urbana_Total
O estado SE tem 0 valores nulos para o indicador Urbana_Total
O estado BA tem 0 valores nulos para o indicador Urbana_Total
O estado

In [17]:
name = 'Urbana_Homem'
name_id = "41"     #População Jovem Urbana: Homem
year_id = "9"      #Ano: 2010

In [18]:
file_name = "Novas/Urbana_Homem.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Urbana_Homem
O estado AC tem 0 valores nulos para o indicador Urbana_Homem
O estado AM tem 0 valores nulos para o indicador Urbana_Homem
O estado RR tem 0 valores nulos para o indicador Urbana_Homem
O estado PA tem 0 valores nulos para o indicador Urbana_Homem
O estado AP tem 0 valores nulos para o indicador Urbana_Homem
O estado TO tem 0 valores nulos para o indicador Urbana_Homem
O estado MA tem 0 valores nulos para o indicador Urbana_Homem
O estado PI tem 0 valores nulos para o indicador Urbana_Homem
O estado CE tem 0 valores nulos para o indicador Urbana_Homem
O estado RN tem 0 valores nulos para o indicador Urbana_Homem
O estado PB tem 0 valores nulos para o indicador Urbana_Homem
O estado PE tem 0 valores nulos para o indicador Urbana_Homem
O estado AL tem 0 valores nulos para o indicador Urbana_Homem
O estado SE tem 0 valores nulos para o indicador Urbana_Homem
O estado BA tem 0 valores nulos para o indicador Urbana_Homem
O estado

In [19]:
for item in listaUF:
    file_name = "Urbana_Homem/" + item +  "_Urbana_Homem.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Urbana_Homem
O estado AC tem 0 valores nulos para o indicador Urbana_Homem
O estado AM tem 0 valores nulos para o indicador Urbana_Homem
O estado RR tem 0 valores nulos para o indicador Urbana_Homem
O estado PA tem 0 valores nulos para o indicador Urbana_Homem
O estado AP tem 0 valores nulos para o indicador Urbana_Homem
O estado TO tem 0 valores nulos para o indicador Urbana_Homem
O estado MA tem 0 valores nulos para o indicador Urbana_Homem
O estado PI tem 0 valores nulos para o indicador Urbana_Homem
O estado CE tem 0 valores nulos para o indicador Urbana_Homem
O estado RN tem 0 valores nulos para o indicador Urbana_Homem
O estado PB tem 0 valores nulos para o indicador Urbana_Homem
O estado PE tem 0 valores nulos para o indicador Urbana_Homem
O estado AL tem 0 valores nulos para o indicador Urbana_Homem
O estado SE tem 0 valores nulos para o indicador Urbana_Homem
O estado BA tem 0 valores nulos para o indicador Urbana_Homem
O estado

In [20]:
name = 'Urbana_Mulher'
name_id = "42"     #População Jovem Urbana: Mulher
year_id = "9"      #Ano: 2010

In [21]:
file_name = "Novas/Urbana_Mulher.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Urbana_Mulher
O estado AC tem 0 valores nulos para o indicador Urbana_Mulher
O estado AM tem 0 valores nulos para o indicador Urbana_Mulher
O estado RR tem 0 valores nulos para o indicador Urbana_Mulher
O estado PA tem 0 valores nulos para o indicador Urbana_Mulher
O estado AP tem 0 valores nulos para o indicador Urbana_Mulher
O estado TO tem 0 valores nulos para o indicador Urbana_Mulher
O estado MA tem 0 valores nulos para o indicador Urbana_Mulher
O estado PI tem 0 valores nulos para o indicador Urbana_Mulher
O estado CE tem 0 valores nulos para o indicador Urbana_Mulher
O estado RN tem 0 valores nulos para o indicador Urbana_Mulher
O estado PB tem 0 valores nulos para o indicador Urbana_Mulher
O estado PE tem 0 valores nulos para o indicador Urbana_Mulher
O estado AL tem 0 valores nulos para o indicador Urbana_Mulher
O estado SE tem 0 valores nulos para o indicador Urbana_Mulher
O estado BA tem 0 valores nulos para o indicador Urbana

In [22]:
for item in listaUF:
    file_name = "Urbana_Mulher/" + item +  "_Urbana_Mulher.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Urbana_Mulher
O estado AC tem 0 valores nulos para o indicador Urbana_Mulher
O estado AM tem 0 valores nulos para o indicador Urbana_Mulher
O estado RR tem 0 valores nulos para o indicador Urbana_Mulher
O estado PA tem 0 valores nulos para o indicador Urbana_Mulher
O estado AP tem 0 valores nulos para o indicador Urbana_Mulher
O estado TO tem 0 valores nulos para o indicador Urbana_Mulher
O estado MA tem 0 valores nulos para o indicador Urbana_Mulher
O estado PI tem 0 valores nulos para o indicador Urbana_Mulher
O estado CE tem 0 valores nulos para o indicador Urbana_Mulher
O estado RN tem 0 valores nulos para o indicador Urbana_Mulher
O estado PB tem 0 valores nulos para o indicador Urbana_Mulher
O estado PE tem 0 valores nulos para o indicador Urbana_Mulher
O estado AL tem 0 valores nulos para o indicador Urbana_Mulher
O estado SE tem 0 valores nulos para o indicador Urbana_Mulher
O estado BA tem 0 valores nulos para o indicador Urbana